In [1]:
%%capture
import os
if "COLAB_" in "".join(os.environ.keys()):
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
if 'HF_TOKEN' not in os.environ:
  os.environ['HF_TOKEN'] = input("Enter HF Token:").strip()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

LLAMA = {
    "src_model": "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "finetuned_model": "saisasanky/Llama-3.1-8B-Instruct-bnb-4bit-aish",
    "gguf_model": "saisasanky/Llama-3.1-8B-Instruct-bnb-4bit-aish-gguf",
    "chat_template": "llama-3.1",
    "instruction_template": "<|start_header_id|>user<|end_header_id|>\n\n",
    "response_template": "<|start_header_id|>assistant<|end_header_id|>\n\n",
}
QWEN = {
    "src_model": "unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit",
    "finetuned_model": "saisasanky/Qwen2.5-Coder-14B-Instruct-bnb-4bit-aish",
    "gguf_model":  "saisasanky/Qwen2.5-Coder-14B-Instruct-bnb-4bit-aish-gguf",
    "chat_template": "qwen-2.5",
    "instruction_template": "<|im_start|>user\n",
    "response_template": "<|im_start|>assistant\n",
}

model_config = LLAMA

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_config["src_model"],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = model_config["chat_template"],
)

from datasets import load_dataset
dataset = load_dataset("westenfelder/NL2SH-ALFA", "train", split = "train").shuffle()


In [ ]:
# prompt: user apply_chat_template on dataset where each row is similar to
# {'nl': 'show the free space on all filesystems', 'bash': 'df -h'}

def formatting_prompts_func(examples):
    nls = examples['nl']
    bashs = examples['bash']

    texts = [tokenizer.apply_chat_template(
        [{"role": "system", "content": "You are an assistant that provides exact bash command for given input"},
         {"role": "user", "content": nl},
         {"role": "assistant", "content": bash}],
        tokenize=False, add_generation_prompt=False)
            for nl,bash in zip(nls,bashs)]
    return {'text': texts}

dataset = dataset.map(formatting_prompts_func, batched=True)
dataset[0]['text']

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
dataset[30]["text"]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 32,
        warmup_steps = 10,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 200,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = model_config["instruction_template"],
    response_part = model_config["response_template"],
)

In [ ]:
tokenizer.decode(trainer.train_dataset[5]['input_ids'])

In [ ]:
space = tokenizer(" ",add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]['labels']])

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = model_config["chat_template"],
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user","content": "Command to List all the contents in the current folder"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors = "pt",
).to("cuda")
outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

In [ ]:
model.push_to_hub(model_config["finetuned_model"], token = os.environ['HF_TOKEN'])
tokenizer.push_to_hub(model_config["finetuned_model"], token = os.environ['HF_TOKEN'])

In [ ]:
#!(cd llama.cpp; cmake -B build;cmake --build build --config Release)

In [ ]:
model.push_to_hub_gguf(
    model_config["gguf_model"],
    tokenizer,
    token = os.environ['HF_TOKEN'],
    quantization_method = 'q8_0'
)